In [ ]:
# Setup script for the colab environment
if 'google.colab' in str(get_ipython()):
    # Install nerfbaselines and conda to google colab
    !pip install -q git+https://github.com/nerfbaselines/nerfbaselines.git

The viewer is implemented in the {py:class}`nerfbaselines.viewer.Viewer` class. The class
is a context manager and it has to be entered first (which spawns the background process handling HTTP requests).
In this tutorial, we will make use of {py:class}`contextlib.ExitStack` to open the context manager across different
Notebook cells. After the viewer is created, we will call `viewer.show_in_notebook` which opens an iframe containing
the viewer. There is also a button {octicon}`link-external` to open the viewer in a new tab.

In [ ]:
from contextlib import ExitStack
from nerfbaselines.datasets import load_dataset

# We will use ExitStack to keep context open across multiple cells
stack = ExitStack()

# Model can be None (in that case we will only visualize dataset)
model = None

# Create the train and test datasets. Note, that we do not need 
# to load the features, as the images will be automatically loaded
# and cached when they are required by the viewer in the background
# process. This speeds up the viewer load time.
train_dataset = load_dataset("external://blender/lego", split="train", load_features=False)
test_dataset = load_dataset("external://blender/lego", split="train", load_features=False)

In [ ]:
# First, we prepare the method to render. In this example, we load an existing
# checkpoint from the official nerfbaselines repository. However, this requires
# the method to be installed, so we skip it to save time. If you want to see the actual rendering, 
# Please run this cell (it can take long time to install the backend on the first run).
# However, you can also skip this cell and only visualize the dataset.
from nerfbaselines import load_checkpoint

if 'google.colab' in str(get_ipython()):
    # Install conda to google colab
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !nerfbaselines install --method gsplat --backend conda

# Load the checkpoint
checkpoint = "https://huggingface.co/nerfbaselines/nerfbaselines/resolve/main/gsplat/blender/lego.zip/checkpoint"
model, nb_info = stack.enter_context(load_checkpoint(checkpoint, backend="conda"))

In [11]:
from nerfbaselines.viewer import Viewer

# Initialize the viewer
viewer = stack.enter_context(Viewer(
    train_dataset=train_dataset, 
    test_dataset=test_dataset, 
    model=model))

# The `show_in_notebook` function opens the iframe inside jupyter/colab.
# After running the function, the viewer will already be visible,
# But it will not be able to render the images yet. In order,
# to handle the rendering requests, we need to call `viewer.run()`
viewer.show_in_notebook()

# Finally, we run `viewer.run()` which waits in loop for new render requests 
# and handles them. We recommend to call the function in a separate cell
# as it can be stopped without stopping the viewer itself. This is useful
viewer.run()


![images/viewer-colab.jpg](images/viewer-colab.jpg)
